In [ ]:
!pip install langchain_core==0.3.75 langchain_huggingface==0.3.1 langchain-community sentence-transformers
!pip install torch torchvision --upgrade

In [5]:
!pip install ibm-watson-studio-lib 

In [26]:
import os
# import toml

from pymilvus import connections
from pymilvus import Collection, utility, connections, FieldSchema, CollectionSchema, DataType

import ibm_boto3
from ibm_botocore.client import Config, ClientError

from langchain_huggingface import HuggingFaceEmbeddings  # Updated import

from dotenv import load_dotenv

# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='c0685463-22dc-4c40-a870-7b47b3dd01be', project_access_token='p-2+hkUcpNdzw5q/8owbqZ+89Q==;NYGAhLMdsfR8Xmx3KeaJ8A==:QLXIs7rOqdY7iUXl6rm//Tmz1x6zD1bARUhhr8pOoQHlbi59Fz+y0AqF9IU3YQX8uaTAM1p19Ei5DpCdC3LfeLCGaEnsaFgswg==')
pc = project.project_context

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space({'token':'p-2+hkUcpNdzw5q/8owbqZ+89Q==;NYGAhLMdsfR8Xmx3KeaJ8A==:QLXIs7rOqdY7iUXl6rm//Tmz1x6zD1bARUhhr8pOoQHlbi59Fz+y0AqF9IU3YQX8uaTAM1p19Ei5DpCdC3LfeLCGaEnsaFgswg=='})


## Load connections information from the project

In [27]:
# list your connections
wslib.list_connections()

[{'name': 'cos-3100014vn6',
  'description': None,
  'asset_id': '2bda8a2a-df89-472d-ade3-72b788943b52',
  'asset_type': 'connection',
  'tags': None},
 {'name': 'milvus-inst-01',
  'description': None,
  'asset_id': 'caae5f3c-72e2-495f-a6e1-229e0f3ef0e3',
  'asset_type': 'connection',
  'tags': None}]

In [28]:
# make sure you use the right connection name for presto
milvus_conn = wslib.get_connection('milvus-inst-01')
cos_conn = wslib.get_connection('cos-3100014vn6')

## Load env.txt file with configuration

In [29]:
with open('.env_all', 'wb') as env_file:
    env_file.write(wslib.load_data('env.txt').read())
# environmental variables store credentials and configuration
load_dotenv('.env_all')

True

## COS Resource

In [51]:
# Constants for IBM COS values
COS_ENDPOINT = f"https://{cos_conn['url']}"
COS_API_KEY_ID = cos_conn['api_key']
COS_INSTANCE_CRN = cos_conn['resource_instance_id']

### make sure that COS_ENDPOINT is the same as for buckets and the same for all buckets, if not -> replace

In [12]:
print(COS_ENDPOINT)
# COS_ENDPOINT = "enter https-prepended endpoint and uncomment"

https://s3.ca-tor.cloud-object-storage.appdomain.cloud


In [52]:
# Create client
cos_resource = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)


## For text splitting and embedding

In [74]:
# COS buckets 
INPUT_BUCKET="inputdocs"
COS_FOLDER = "inputdoc"
# # parameters for milvus ingestion 
# SIMILARITY_METRIC="L2"
# SENTENCE_TRANSFORMER = "sentence-transformers/all-MiniLM-L6-v2"
# TEXT_SPLITTER_CHUNK_SIZE=1000
# TEXT_SPLITTER_CHUNK_OVERLAP=200
# TEXT_SPLITTER_SEPARATORS='[" \n", "\n"]'
# TEXT_REPLACEMENTS='{"✔": "ok"}'
# TEXT_SPLITTER_TYPE="RecursiveCharacterTextSplitter"

In [33]:
embeddings_model_name = os.getenv("SENTENCE_TRANSFORMER")
chunk_size = int(os.getenv("TEXT_SPLITTER_CHUNK_SIZE"))
chunk_overlap = int(os.getenv("TEXT_SPLITTER_CHUNK_OVERLAP"))

# TEXT - SPLITTER RELATED
text_splitter_type = os.getenv("TEXT_SPLITTER_TYPE")
# to define text splitter incl. text splitter separators (defined in the environment variables)
# json string
text_splitter_separators = os.getenv("TEXT_SPLITTER_SEPARATORS")
text_replacements = os.getenv("TEXT_REPLACEMENTS")

similarity_metric = os.getenv("SIMILARITY_METRIC")

INPUT_BUCKET = os.environ["INPUT_BUCKET"]

In [54]:
INPUT_BUCKET

'inputdocs'

In [42]:
# embeddings function - TO SELECT FUNCTION and PARAMETERS
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

## For milvus connection

In [43]:
# Authentication for Milvus within wx.data
host = milvus_conn["host"]
port = milvus_conn["port"]
password = milvus_conn["password"]
user = milvus_conn["username"]

mv_collection = os.environ["MV_COLLECTION_NAME"]


In [44]:
connection_args={
               'host':host,
               'port':port,
               'user': user,
               'password': password,
                'secure': True
}

In [71]:
RecursiveCharacterTextSplitter

langchain_text_splitters.character.RecursiveCharacterTextSplitter

# 1. Load and split documents

In [78]:
import json
import os
import posixpath

from ibm_botocore.client import ClientError

from langchain.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    TextLoader,
    UnstructuredHTMLLoader,
)
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

# dictionary with text splitter classes
text_splitter_type_dict = {
    "CharacterTextSplitter": CharacterTextSplitter,
    "RecursiveCharacterTextSplitter": RecursiveCharacterTextSplitter,
}

# dictionary with available document loaders
_document_loaders_dict = {
    "docx": Docx2txtLoader,
    "html": UnstructuredHTMLLoader,
    "pdf": PyPDFLoader,
    "csv": TextLoader,
    "txt": TextLoader,
}

def get_bucket_contents(cur_conn, bucket_name):
    """
    function to get the list of files from the given bucket
    """
    items_list = []
    print("Retrieving bucket contents from:{0}".format(bucket_name))
    try:
        files = cur_conn.Bucket(bucket_name).objects.all()
        print(files)
        for file in files:
            items_list.append("{0}".format(file.key))
    except ClientError as be:
        print("CLIENT ERROR: {0}".format(be))
    except Exception as e:
        print("Unable to retrieve bucket contents: {0}".format(e))
    print(items_list)
    return items_list

def get_item(cur_conn, bucket_name, item_name):
    """
    get file contents of a particular file:
    - based on bucket_name and item_name

    """
    print(
        "Retrieving item from bucket: {0}, key: {1}".format(
            bucket_name, item_name
        )
    )
    try:
        file = cur_conn.Object(bucket_name, item_name).get()
        print("File Contents retrieved")
        print(file)
        return file
    except ClientError as be:
        print("CLIENT ERROR: {0}".format(be))
    except Exception as e:
        print("Unable to retrieve file contents: {0}".format(e))

def _save_doc_linux(cur_conn, bucket_name, doc_path, tmp_location):
    """Save file from COS to temp location locally"""
    bytes_obj = get_item(cur_conn, bucket_name, doc_path)["Body"].read()
    with open(tmp_location, "wb") as f:
        f.write(bytes_obj)


def _replace_characters(text_replacements, init_text: str) -> str:
    """
    Replaces characters in the init_text based on text_replacements that contains mapping {"old_char": "new_char"}.
    Returns text with the replaced character
    """
    replace_json_dict = json.loads(text_replacements)
    output_text = init_text
    for old_char, new_char in replace_json_dict.items():
        output_text = output_text.replace(old_char, new_char)
    return output_text


def load_split_cos_docs(
    cur_conn,
    bucket_name,
    chunk_size,
    chunk_overlap,
    text_splitter_type,
    text_splitter_separators,
    text_replacements,
):
    """
    Load and split txt, pdf and doc documents from bucket on COS, where:
    - cur_conn is connection established to COS
    - bucket_name is the name of the bucket with documents to load and split
    - chunk size is the size of text to split by
    - chunk overlap is the number of characters to overlap between chunks
    By default it will use original language, to translate to english you need to set translate to True
    """
    # list to save all docs into a list
    split_loaded_docs = list()
    temp_folder = os.path.join(".", "temp")

    if text_splitter_separators is not None and text_splitter_separators != "":
        text_splitter_separators = json.loads(text_splitter_separators, strict=False)

    # location for temp files
    bucket_contents = get_bucket_contents(cur_conn, bucket_name)
    print(bucket_contents)
    # defining the splitter
    if text_splitter_type in text_splitter_type_dict:
        print(f"Using {text_splitter_type}")
        # free text
        if (
            isinstance(text_splitter_separators, list)
            and len(text_splitter_separators) > 0
        ):
            text_splitter = text_splitter_type_dict[text_splitter_type](
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                separators=text_splitter_separators,
            )
            print(
                f"Specified separators for the text splitter {text_splitter_separators}"
            )
        else:
            text_splitter = text_splitter_type_dict[text_splitter_type](
                chunk_size=chunk_size, chunk_overlap=chunk_overlap
            )
            print(
                "Separators for text splitters are either not specified or specified incorrectly"
            )
    else:
        print(f"Text splitter of type {text_splitter_type} is not supported")

    # going through files
    print("Looping through bucket contents")
    for doc_path_num, doc_path in enumerate(bucket_contents):
        # to check only files within project sources
        sources_folder_path = posixpath.join(COS_FOLDER)
        if sources_folder_path not in doc_path:
            continue
        print(f"Processing {doc_path}")
        cur_doc_format = doc_path.split(".")[-1].lower()
        # to create temp folder if it doesn't exist
        if not os.path.exists(temp_folder):
            os.mkdir(temp_folder)
        tmp_location = os.path.join(temp_folder, doc_path.split("/")[-1])
        print(f"Attempting to load {doc_path}")
        # check for formatting
        if cur_doc_format in _document_loaders_dict:
            # to load document
            _save_doc_linux(cur_conn, bucket_name, doc_path, tmp_location)
            doc_loader = _document_loaders_dict[cur_doc_format](tmp_location)
            loaded_docs = doc_loader.load()
        else:
            logger.warning(
                f"Data in {doc_path} is not supported, accepted formats are {_document_loaders_dict.keys()}"
            )
            continue
        print(f"Successfully loaded {doc_path}")

        # to change metadata of loaded documents to include only filename for the source
        # replace strings based on the input json
        for loaded_doc in loaded_docs:
            # to change metadata of loaded documents to include only filename for the source
            loaded_doc.metadata["source"] = doc_path.split("/")[-1]
            # to replace characters in page_content according to dict stored in TEXT_REPLACEMENTS
            if text_replacements != "" and text_replacements is not None:
                loaded_doc.page_content = _replace_characters(
                    text_replacements, loaded_doc.page_content
                )
        # to split documents
        print(f"Splitting docs loaded from {doc_path}")
        splitted_docs = text_splitter.split_documents(loaded_docs)
        # to add current document to total list
        split_loaded_docs.extend(splitted_docs)
        if os.path.exists(tmp_location):
            os.remove(tmp_location)
            print(f"File {tmp_location} was removed")
    # after loop through COS documents ended
    print("Documents from Cloud Object Storage are loaded and splitted")
    return {"splitted_loaded_docs": split_loaded_docs}


In [79]:
splitted_docs = load_split_cos_docs(
        cos_resource, INPUT_BUCKET, chunk_size, chunk_overlap, text_splitter_type, text_splitter_separators, text_replacements
        )

Retrieving bucket contents from:inputdocs
s3.Bucket.objectsCollection(s3.Bucket(name='inputdocs'), s3.ObjectSummary)
['Simmons-Bank-Code-of-Ethics.pdf', 'inputdoc/pie_recipe.pdf', 'paper_flowers.pdf']
['Simmons-Bank-Code-of-Ethics.pdf', 'inputdoc/pie_recipe.pdf', 'paper_flowers.pdf']
Using RecursiveCharacterTextSplitter
Specified separators for the text splitter [' \n', '\n']
Looping through bucket contents
Processing inputdoc/pie_recipe.pdf
Attempting to load inputdoc/pie_recipe.pdf
Retrieving item from bucket: inputdocs, key: inputdoc/pie_recipe.pdf
File Contents retrieved
{'ResponseMetadata': {'RequestId': '31b0ff5b-6c70-43a0-8c51-b8d7e7822cfc', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 25 Oct 2025 11:20:29 GMT', 'x-clv-request-id': '31b0ff5b-6c70-43a0-8c51-b8d7e7822cfc', 'server': 'Cleversafe', 'x-clv-s3-version': '2.5', 'accept-ranges': 'bytes', 'x-amz-request-id': '31b0ff5b-6c70-43a0-8c51-b8d7e7822cfc', 'etag': '"247a65c91848b285b08cf32fe2c55ea8"', 'conte

In [80]:
sources = []
pages = []
docs = []
for splitted_doc in splitted_docs['splitted_loaded_docs']:
    sources.append(splitted_doc.metadata['source'])
    pages.append(splitted_doc.metadata['page'])
    docs.append(splitted_doc.page_content)

In [81]:
splitted_docs

{'splitted_loaded_docs': [Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2021-02-05T12:37:19-08:00', 'author': 'Stella, Gavin (RNA/MK-MAB)', 'moddate': '2021-02-05T12:37:19-08:00', 'source': 'pie_recipe.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Curtis Stone’s Meat Pie Recipe \n \nServes 6 | Prep Time: 30 minutes | Cook Time: 1 hour and 45 minutes \n \nIngredients: \n \n\uf0b7 1 ¼ cup extra-virgin olive oil  \n\uf0b7 1 small yellow onion, finely chopped  \n\uf0b7 1 celery stalk, finely chopped  \n\uf0b7 1 small carrot, finely chopped  \n\uf0b7 2 ounces prosciutto di Parma (about 5 \nslices), finely chopped  \n\uf0b7 1 ¾ pounds ground beef chuck \n\uf0b7 Kosher salt and freshly ground black \npepper  \n\uf0b7 3 ounces chicken livers (about 3), \nrinsed, cleaned, and finely chopped  \n\uf0b7 2 tablespoons tomato paste  \n\uf0b7 ⅛ teaspoon ground allspice  \n\uf0b7 ½ cup dry white wine \n\uf0b7 ½ cup who

# 2. Milvus

## Connect

In [82]:
connections.connect(
alias='default',
**connection_args
)

In [83]:
utility.has_collection(mv_collection)

False

## To drop collection if exists

In [84]:
if utility.has_collection(mv_collection): # check if collection exists
    utility.drop_collection(mv_collection)

## To create a new collection
> you can update schema naming to correspond to your requirements

In [85]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="text_embedding", dtype=DataType.FLOAT_VECTOR, dim=384),  # Assuming 384-dim vectors
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="page", dtype=DataType.INT64),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),  # Storing original content as well
]
schema = CollectionSchema(fields)
collection = Collection(name=mv_collection, schema=schema)

In [86]:
print(collection.schema.fields)

[{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'text_embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 500}}, {'name': 'page', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}]


## To insert documents into collection, create index and load data into memory

In [87]:
# Convert documents into vector embeddings
doc_embeddings = embeddings.embed_documents(docs)

# Insert documents and embeddings into Milvus
collection.insert([doc_embeddings, sources, pages, docs]) 

(insert count: 6, delete count: 0, upsert count: 0, timestamp: 461738156283920387, success count: 6, err count: 0

In [88]:
# create index
index_params = {
    "index_type": "IVF_FLAT",  # Can also use "IVF_PQ", "HNSW", etc.
    "metric_type": similarity_metric,       # L2 for Euclidean distance, or use "IP" for Inner Product
    "params": {"nlist": 110},  # nlist is a hyperparameter for clustering / corresponds to cca 800 vectors
}

collection.create_index(field_name="text_embedding", index_params=index_params)

Status(code=0, message=)

In [89]:
# Load data into memory (optional but recommended for larger datasets)
collection.load()

## To perform semantic search

In [90]:
cur_query="top ingredients in Meat Pie Recipe"
search_params = {"metric_type": similarity_metric, "params": {"nprobe": 10}}
query_result = collection.search(
    [embeddings.embed_query(cur_query)], 
    "text_embedding", 
    search_params, 
    limit=5, 
    output_fields=["title", "page", "text"])
# Print the query result
for entity in query_result[0]:
    print(entity)

id: 461692709396379109, distance: 0.5376728773117065, entity: {'title': 'pie_recipe.pdf', 'page': 0, 'text': 'Curtis Stone’s Meat Pie Recipe \n \nServes 6 | Prep Time: 30 minutes | Cook Time: 1 hour and 45 minutes \n \nIngredients: \n \n\uf0b7 1 ¼ cup extra-virgin olive oil  \n\uf0b7 1 small yellow onion, finely chopped  \n\uf0b7 1 celery stalk, finely chopped  \n\uf0b7 1 small carrot, finely chopped  \n\uf0b7 2 ounces prosciutto di Parma (about 5 \nslices), finely chopped  \n\uf0b7 1 ¾ pounds ground beef chuck \n\uf0b7 Kosher salt and freshly ground black \npepper  \n\uf0b7 3 ounces chicken livers (about 3), \nrinsed, cleaned, and finely chopped  \n\uf0b7 2 tablespoons tomato paste  \n\uf0b7 ⅛ teaspoon ground allspice  \n\uf0b7 ½ cup dry white wine \n\uf0b7 ½ cup whole milk  \n\uf0b7 2 tablespoons all-purpose flour  \n\uf0b7 3 cups low-sodium beef broth \n \nTo assemble the pies: \n\uf0b7 All-purpose flour, for dusting \n\uf0b7 Buttery Pastry Dough (see recipe \nbelow), shaped into 12